In [1]:
import scipy.io
import numpy as np
import datetime 
import ephem
import random
from scipy import spatial
import matplotlib.pyplot as plt
%matplotlib
from astropy.table import Table
import astropy.units as u
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
import astropy
from astropy.time import Time
from astropy.coordinates import EarthLocation
import sunpy.coordinates
from datetime import datetime, timezone
from math import atan, atan2, degrees, floor, pi, radians, sin, sqrt
from skyfield.api import earth, JulianDate, now, sun
data=scipy.io.loadmat('BNS-sources-trial-10saleem.mat')['data'] 
#sourceparameters
DL=data[:,0]
theta=data[:,1]
phi=data[:,2]

Using matplotlib backend: MacOSX


In [2]:
lat = np.pi/2 - theta
long = phi

In [3]:
from astropy.time import Time
time1 = Time('2022-01-01T00:00:00') 
time2 = Time('2023-01-01T00:00:00')
#%time homotime = Time(np.linspace(time1.jd, time2.jd, len(theta)), format="jd")
randomtime = Time(np.random.uniform(low=time1.jd, high=time2.jd, size=len(theta)), format="jd")

In [18]:
#print(-5 %360)
#print(370 % 360)

355
10


In [4]:
def radectolatlong(Ra,Dec,time): #input in degrees
    """
    Take the RA, Dec of GW trigger from the table and convert them to  longitude, latitude
    RA, Dec should be floating point numbers (not astropy.coordinates)
    time should be a astropy.Time.time object
    Output is in floating point numbers (not astropy.quantity or astropy.coordinates)
    """
    ra = np.deg2rad(Ra)
    dec = np.deg2rad(Dec)
    time_sid = (time.sidereal_time('mean','greenwich')).rad
    lat = dec
    long = ra - time_sid 
    Lat = np.rad2deg(lat) %360
    Long = np.rad2deg(long)
    return Lat,Long #output in degrees

def latlongtoradec(Lat,Long,time): #input in degrees
    lat = np.deg2rad(Lat)
    long = np.deg2rad(Long)
    time_sid = (time.sidereal_time('mean','greenwich')).rad
    DEC = lat
    RA = long + time_sid
    ra = np.rad2deg(RA) %360     
    dec = np.rad2deg(DEC)
    return ra,dec #output in degree

def getxyz(theta,phi):
    """
    Convert cartesian coordinates into theta phi for ANY FRAME OF REFERENCE
    Theta ranges from blah to blah, and is a floating point number in RADIANS
    Phi ranges from blah to blah, and is a floating point number in RADIANS
    X,y z are floating point numbers - and make a unit vector
    """
    assert theta.unit == u.rad, "This function needs astropy.units.radians as input"
    #if n==1:
        #lat, long
    theta = np.pi/2 - theta
    x=np.sin(theta)*np.cos(phi)
    y=np.sin(theta)*np.sin(phi)
    z=np.cos(theta)
    return x,y,z
def getnorm(vec):
    norm = np.linalg.norm(vec)
    #print (norm)
    return vec/norm

In [23]:
np.sin(90 * u.deg), np.sin(np.pi/2 * u.rad)

(<Quantity 1.>, <Quantity 1.>)

In [5]:
import pandas as pd

In [6]:
RAs = np.empty(len(theta),dtype=object)
DECs = np.empty(len(theta),dtype=object)
times = np.empty(len(theta),dtype=object)

In [7]:
for i in range(0,len(theta)):
    LLRD = latlongtoradec(lat[i],long[i],randomtime[i])
    RAs[i] = LLRD[0]
    DECs[i] = LLRD[1]
    times[i] = randomtime[i]

In [8]:
df = pd.DataFrame(RAs,columns={"RA in hour angle"})
df['DEC in degree'] = pd.DataFrame(DECs)
df['Time in isot'] = pd.DataFrame(randomtime.isot)
df['Time in jd'] = pd.DataFrame(randomtime.jd)

In [9]:
df.to_csv('/Users/jinaypatel/Desktop/Daksha/Events(triggers).csv', sep=',', encoding='utf-8')

In [10]:
df['lat'] = pd.DataFrame(lat)
df['long'] = pd.DataFrame(long)

In [11]:
homotime = np.sort(randomtime.isot)

In [12]:
latlongtoradec(0,0,Time('2019-03-20T09:58:00'))

(327.14985403573723, 0.0)

In [13]:
randomtime.iso[0]

'2022-12-06 18:26:24.558'

In [14]:
Time('2019-04-15T00:58:00').sidereal_time('mean','greenwich').rad

3.794467313623034

In [15]:
radectolatlong(0,0,Time('2019-03-20T18:49:47'))

(0.0, -100.45967831899189)